In [ ]:
import cv2
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import face_recognition
from threading import Thread

# Paths
model_save_path = "efficientnet_b2_emotion_model.pth"  # Path to the saved model

# Emotion categories
emotion_classes = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the emotion model
def load_model(model_path):
    model = models.efficientnet_b2(pretrained=False)
    model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)  # Adjust for grayscale
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.6),  # Match the dropout used during training
        nn.Linear(model.classifier[1].in_features, len(emotion_classes)),
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model


# Load the trained model
model = load_model(model_save_path)


# Image preprocessing function
def preprocess_face(face_image):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Ensure grayscale
        transforms.Resize((224, 224)),               # Resize to model input size
        transforms.ToTensor(),                       # Convert to tensor
        transforms.Normalize(mean=[0.485], std=[0.229]),  # Normalize for grayscale images
    ])
    pil_image = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
    processed_image = transform(pil_image)
    processed_image = processed_image.unsqueeze(0)  # Add batch dimension
    return processed_image.to(device)


# Prediction function
def predict_emotion(face_image):
    # Preprocess the input image
    processed_image = preprocess_face(face_image)

    # Perform prediction
    with torch.no_grad():
        outputs = model(processed_image)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class_idx = torch.argmax(probabilities, dim=1).item()
        predicted_class = emotion_classes[predicted_class_idx]
        confidence = probabilities[0, predicted_class_idx].item()

    return predicted_class, confidence


# Face detection and emotion prediction with OpenCV rendering
def run_live_emotion_detection():
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Cannot access webcam.")
        return

    print("Press 'q' to quit the application.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Cannot read frame from webcam.")
            break

        # Resize the frame for faster face detection
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the resized frame
        face_locations = face_recognition.face_locations(rgb_small_frame)

        # Scale face locations back to original frame size
        face_locations = [(top * 4, right * 4, bottom * 4, left * 4) for (top, right, bottom, left) in face_locations]

        # Draw rectangles around detected faces and predict emotions
        for (top, right, bottom, left) in face_locations:
            face_image = frame[top:bottom, left:right]

            # Skip prediction if the face is too small
            if face_image.shape[0] > 0 and face_image.shape[1] > 0:
                emotion, confidence = predict_emotion(face_image)
                print(f"Emotion Detected: {emotion} (Confidence: {confidence:.2f})")

                # Draw the rectangle
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

                # Display the emotion label
                label = f"{emotion} ({confidence:.2f})"
                cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Display the frame
        cv2.imshow("Live Emotion Detection", frame)

        # Press 'q' to quit the application
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# Run the emotion detection
if __name__ == "__main__":
    run_live_emotion_detection()


c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Tuf\AppData\Local\Temp\ipykernel_12388\2025366839.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future relea

Press 'q' to quit the application.
Emotion Detected: Neutral (Confidence: 0.61)
Emotion Detected: Neutral (Confidence: 0.51)
Emotion Detected: Neutral (Confidence: 0.76)
Emotion Detected: Neutral (Confidence: 0.69)
Emotion Detected: Neutral (Confidence: 0.69)
Emotion Detected: Neutral (Confidence: 0.43)
Emotion Detected: Neutral (Confidence: 0.80)
Emotion Detected: Neutral (Confidence: 0.72)
Emotion Detected: Neutral (Confidence: 0.52)
Emotion Detected: Neutral (Confidence: 0.76)
Emotion Detected: Neutral (Confidence: 0.41)
Emotion Detected: Neutral (Confidence: 0.72)
Emotion Detected: Neutral (Confidence: 0.52)
Emotion Detected: Neutral (Confidence: 0.82)
Emotion Detected: Neutral (Confidence: 0.71)
Emotion Detected: Neutral (Confidence: 0.48)
Emotion Detected: Neutral (Confidence: 0.64)
Emotion Detected: Neutral (Confidence: 0.71)
Emotion Detected: Neutral (Confidence: 0.79)
Emotion Detected: Neutral (Confidence: 0.49)
Emotion Detected: Neutral (Confidence: 0.49)
Emotion Detected: Ne

KeyboardInterrupt: 

: 